# Sequential Model Demo Full CNN

## 00. Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
import sys
sys.path.append("../input/cars-numpy/")

In [ ]:
from src.activations.relu import ReluLayer
from src.layers.pooling import AvgPoolLayer, MaxPoolLayer
from src.activations.softmax import SoftmaxLayer
from src.layers.dense import DenseLayer
from src.layers.flatten import FlattenLayer
from src.layers.convolution import ConvLayer2D
from src.layers.dw_conbolution import DWConvLayer2d
from src.layers.dropout import DropoutLayer
from src.models.sequential import SequentialModel
from src.utils.core import convert_categorical2one_hot, convert_prob2categorical
from src.utils.metrics import softmax_accuracy
from src.optimizers.nadam import NAdam
from src.utils.plots import lines
from src.utils.core import generate_batches

## 01. Settings

In [ ]:
# number of samples in the train data set
N_TRAIN_SAMPLES = 8144
# number of samples in the test data set
N_TEST_SAMPLES = 1000
# number of samples in the validation data set
N_VALID_SAMPLES = 300
# number of classes
N_CLASSES = 197
# image size
IMAGE_SIZE = 224

In [ ]:
trainX = np.load('../input/cars-numpy/cars_train_annos.npz')['arr_0']
trainY = np.load('../input/cars-numpy/cars_train_annos.npz')['arr_1']
testX = np.load('../input/cars-numpy/cars_test_annos.npz')['arr_0']
testY = np.load('../input/cars-numpy/cars_test_annos.npz')['arr_1']

## 02. Build data set

In [ ]:
print("trainX shape:", trainX.shape)
print("trainY shape:", trainY.shape)
print("testX shape:", testX.shape)
print("testY shape:", testY.shape)

In [ ]:
X_train = trainX[:N_TRAIN_SAMPLES, :, :, :]
y_train = trainY[:N_TRAIN_SAMPLES]

X_valid = testX[N_TEST_SAMPLES:N_TEST_SAMPLES+N_VALID_SAMPLES, :, :, :]
y_valid = testY[N_TEST_SAMPLES:N_TEST_SAMPLES+N_VALID_SAMPLES]

X_test = testX[:N_TEST_SAMPLES, :, :, :]
y_test = testY[:N_TEST_SAMPLES]

In [ ]:
del trainX
del trainY
del testX
del testY

**NOTE:** We need to change the data format to the shape supported by my implementation.

In [ ]:
X_train = (X_train / 255)
# X_train = np.expand_dims(X_train, axis=3)
y_train = convert_categorical2one_hot(y_train)
X_test = (X_test / 255)
# X_test = np.expand_dims(X_test, axis=3)
y_test = convert_categorical2one_hot(y_test)
X_valid = (X_valid / 255)
# X_valid = np.expand_dims(X_valid, axis=3)
y_valid = convert_categorical2one_hot(y_valid)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)

## 03. Build model

In [ ]:
def init_mbconv(t: int, k: int, c: int, n: int, s: int):
    layers = []
    block = []
    tk = t * k
    if s == 1:
        block = [
            ConvLayer2D.initialize(filters=tk, kernel_shape=(1, 1, k), stride=1, padding='same'),
            ReluLayer(),
            DWConvLayer2d.initialize(kernel_shape=(3, 3, tk), stride=s, padding='same'),
            ReluLayer(),
            ConvLayer2D.initialize(filters=c, kernel_shape=(1, 1, tk), stride=1, padding='same')
        ]
    elif s == 2:
        block = [
            ConvLayer2D.initialize(filters=tk, kernel_shape=(1, 1, k), stride=1, padding='same'),
            ReluLayer(),
            DWConvLayer2d.initialize(kernel_shape=(3, 3, tk), stride=s, padding='valid', padding_value=1),
            ReluLayer(),
            ConvLayer2D.initialize(filters=c, kernel_shape=(1, 1, tk), stride=1, padding='same')
        ]
    layers += block
    ck = c * k
    for i in range(n - 1):
        block = [
            ConvLayer2D.initialize(filters=ck, kernel_shape=(1, 1, c), stride=1, padding='same'),
            ReluLayer(),
            DWConvLayer2d.initialize(kernel_shape=(3, 3, ck), stride=1, padding='same'),
            ReluLayer(),
            ConvLayer2D.initialize(filters=c, kernel_shape=(1, 1, ck), stride=1, padding='same')
        ]
        layers += block
    
    return layers


layers = [ConvLayer2D.initialize(filters=32, kernel_shape=(3, 3, 3), stride=2, padding='valid', padding_value=1),
          ReluLayer()]
mbconv_configs = [(1, 32, 16, 1, 1), (6, 16, 24, 2, 2), (6, 24, 32, 3, 2),
                  (6, 32, 64, 4, 2), (6, 64, 96, 3, 1), (6, 96, 160, 3, 2),
                  (6, 160, 320, 1, 1)]
for mbconv_params in mbconv_configs:
    mbconv = init_mbconv(*mbconv_params)
    layers += mbconv

layers += [ConvLayer2D.initialize(filters=1280, kernel_shape=(1, 1, 320), stride=1, padding='same'),
           ReluLayer(),
           AvgPoolLayer((7, 7), stride=1),
           ConvLayer2D.initialize(filters=N_CLASSES, kernel_shape=(1, 1, 1280), stride=1, padding='same'),
           FlattenLayer(),
           SoftmaxLayer()
           ]

optimizer = NAdam(lr=0.003)

model = SequentialModel(
    layers=layers,
    optimizer=optimizer
)

## 04. Train

In [ ]:
model.train(
    x_train=X_train, 
    y_train=y_train, 
    x_test=X_test, 
    y_test=y_test, 
    epochs=10,
    bs=32,
    verbose=True
)

## 05. Predict and examine results

In [ ]:
lines(
    y_1=np.array(model.history["train_acc"]),
    y_2=np.array(model.history["test_acc"]),
    label_1="train",
    label_2="valid",
    title="ACCURACY",
    fig_size=(16,10),
)

In [ ]:
lines(
    y_1=np.array(model.history["train_loss"]),
    y_2=np.array(model.history["test_loss"]),
    label_1="train",
    label_2="valid",
    title="LOSS",
    fig_size=(16,10),
)

In [ ]:
y_hat = np.zeros_like(y_test)
for idx, (x_batch, y_batch) in \
        enumerate(generate_batches(X_test, y_test, 32)):
    x_batch = np.asarray(x_batch)
    y_batch = np.asarray(y_batch)

    y_hat_batch = model.predict(x_batch)
    y_hat_batch = y_hat_batch
    n_start = idx * 32
    n_end = n_start + y_hat_batch.shape[0]
acc = softmax_accuracy(y_hat, y_test)
print("acc: ", acc)

In [ ]:
# del X_train
# del y_train
# del X_valid
# del y_valid
# del X_test
# del y_test
# import pickle 
# with open('model.object', 'wb') as file: 
#     pickle.dump(model, file)

In [ ]:
# y_hat = convert_prob2categorical(y_hat)
# y_valid = convert_prob2categorical(y_valid)

In [ ]:
# df_cm = pd.DataFrame(
#     confusion_matrix(y_valid, y_hat), 
#     range(10), 
#     range(10)
# )
# plt.figure(figsize = (16,16))
# sn.heatmap(df_cm, annot=True, cmap="YlGnBu", linewidths=.5, cbar=False)
# plt.savefig("../viz/cm.png", dpi=100)
# plt.show()